<a href="https://colab.research.google.com/github/abergues/SalsaApp/blob/eero/preprocess_youtube_videos_to_openpose_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pose Detection with OpenPose

Based on code from Colab.  

This notebook uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) to detect multi person poses on a given youtube video.

It is weak on disapering body parts and persons. And there is often jitter when an arm is partially hidden.



### Connect to Google Drive

You need to manually copy the credentials from Google Drive to Colab, when asked.

In [48]:
import os
from os.path import exists, join, basename, splitext
from google.colab import drive
import glob                       # help for finding files
from re import split              # regular expression string splitter
import subprocess                 #subprocess wraps around regular os commands
import shutil                     # used for copying files in the os.

print("When you mount GDrive, you will be asked for permission, so allow it,")
print("copy the key, paste it in the input field in Colab, ")
print("and press Enter. \n")
drive.mount('/gdrive')   

# Google Drive root directory
root_path = "/gdrive/My Drive/DSR/DanceApp"

print("\n", "Working directory is now ", os.getcwd())
print("All files (videos etc) should be placed in ", root_path, "\n")


# Confirm runtime to GPU
! nvcc --version
! nvidia-smi

When you mount GDrive, you will be asked for permission, so allow it,
copy the key, paste it in the input field in Colab, 
and press Enter. 

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).

 Working directory is now  /content
All files (videos etc) should be placed in  /gdrive/My Drive/DSR/DanceApp 

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Mon Nov 29 16:36:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute 

## Install OpenPose 
This is similar to creating an environment, so it takes 10-15 minutes. 


In [49]:
print("Working directory is now ", os.getcwd(), "\n")

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):                                    ## This saves Lots of time when rerunning the whole notebook.
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`



Working directory is now  /content 



In [50]:
print("Working directory is now ", os.getcwd(), "\n")

# ffprobe is needed to find the correct frames per second of the videos downloaded
!pip install ffprobe

from IPython.display import YouTubeVideo


Working directory is now  /content 



# Define which videos to download and process

Setting the global parameters that functions will be using

In [51]:
#                YOUTUBE_ID  , clip_name      , start_mmss, duration
#                              clip_names should be unique and work as folder name
youtube_list= [["Xm8w6L1bFoQ", "Suzie_Q_ToeTap_HookStep_GhettoFab" , "00:10" , "01:30"]
            #   ["9jxMHhuvEyw", "1p_salsa_basic_tutorial"  , "00:07" , "02:20"]
            #   ["vxp5sphsTtI" , "1p_Captain_Salsa_on1", "01:07", "13:30"]
              ]

""" DONE       ["7VA03NNXPVQ", "10_salsa_patterns", "01:54", "17:00"],
               ["BD7oCwXIvow", "6_turn_system"    , "05:38", "05:10"],
               ['e_EcZZS7Psk', "7_salsa_moves_on1"       , "00:59" , "02:10" ],
               ['E6ofN0HpI6Q', "7_inside_turn_variations", "01:03" , "02:22"],       
               ['lkYIrcac4mo',"enchufla_doble", "00:00", "00:30" ], 
               ['cd2ROSr-f_8','enchufla',       "00:15", "00:30" ],
               ['4Q_fnhxsQL4', '13figures',     "00:00", "00:30" ]
"""              
#options for ffmpg
# "Youtube_ID"  defines which video to download
# clip_name is a short unique name that is used for creating a folder etc.
# start_mmss = "00:00"              #-ss option must be given in format mm:ss
# duration   = "00:30"              #-t optionmust be given in format mm:ss


# parameters for single frame images
size_of_single_frame = "640x360"


# the jpgs and json numbering need to match 
save_frames_per_second = 5  # the number of jpgs per second
default_fps = 25     # the default frames per second of the videos, 
                     # which often is wrong  
                     # and needs to be corrected after the download
                     # so that the json and jpg files match each other.

original_working_dir = os.getcwd()   # this could be somewhere much earlier
print("current working directory is ", original_working_dir)

generic_file_name  = os.path.join(root_path, "input_original", "youtube.mp4")
specific_file_name = os.path.join(root_path, "input_original", f"{YOUTUBE_ID}.mp4")

current working directory is  /content


# Define all functions

Because GDrive does not well with file operations, files are usually renamed, instead of copied around. This is much faster.


In [52]:
# import subprocess                 #subprocess wraps around regular os commands
#   # TODO: frames_per_s might need processing. grep keep only numbers. 
# os.chdir(root_path + "/input_original/")
# print(os.listdir())
# !ffprobe -v error -show_entries format=r_frame_rate -of default=noprint_wrappers=1:nokey=1 E6ofN0HpI6Q.mp4
# #frames_per_s  = subprocess.getoutput('ffprobe -v error -show_entries format=r_frame_rate -of default=noprint_wrappers=1:nokey=1 ./input_original/video.mp4')
# print("Number of frames per second: ", frames_per_s)
# print("\n")
# frames_per_s



In [53]:
#   generic_file_name  = os.path.join(root_path, "input_original", "youtube.mp4")
#   specific_file_name = os.path.join(root_path, "input_original", f"{YOUTUBE_ID}.mp4")
#   print(generic_file_name, specific_file_name)
 
#   # there are now two different ways of dealing with filenames. one for ffprobe
#   # that use os.chdir(root_path), and then all the other parts, 
#   # where root_path is not used as working directory.
#   # It is bad practice, but the easy way to make it reliable. $root_path/ did often fail.

#   # skip downloading a youtube video we have already 
#   print(os.getcwd())
#   if os.path.isfile(specific_file_name):
#     print("renaming the file from ", specific_file_name, " to: ", generic_file_name)
#     shutil.copyfile(specific_file_name, generic_file_name)
# # download_youtube()

In [54]:
import os
from pathlib import Path



def prepare_folders_and_delete_outputs():
  # create a folder for openpose outputs
  # requires that clipname is defined
 
  if not exists(os.path.join(root_path, "output_op", clip_name)):
    !mkdir "$root_path/output_op/$clip_name/"
    print("Created a folder for the Openpose outputs : ", root_path, "/output_op/", clip_name)

  # Get rid of old versions before the new ones are created so that there is no 
  # mixup between which sources have been used for the outputs.

  !rm -rf "$root_path/input_original/youtube.mp4"   # the generic video from youtube
  !rm -rf "$root_path/input_original/openpose.avi"  # stickfigures after prosessing with openpose
  !rm -rf "$root_path/input_original/video.mp4"     # the generic video from youtube

  # should be deleted because we process file anew
  print("Deleting old files from the ", clip_name, " folder")
  # create a list of files to delete
  delete_files = glob.glob(os.path.join(root_path, "output_op", clip_name, "*.jpg"))
  #print("jpg: " , delete_files[-3:])
  delete_files.extend(glob.glob(os.path.join(root_path, "output_op", clip_name, "*.jpeg")))
  delete_files.extend(glob.glob(os.path.join(root_path, "output_op", clip_name, "*.json")))
  #print("json: ", delete_files[-3:])
  delete_files.extend(glob.glob(os.path.join(root_path, "output_op", clip_name, "*.mp4")))
  #print("mp4: ", delete_files[-3:])
  delete_files.extend(glob.glob(os.path.join(root_path, "output_op", clip_name, "*.avi")))
  #print("avi: ", delete_files[-1:])
  [os.remove(x) for x in delete_files] 
  
  print("\n Now the folder contains only these files:")
  print(glob.glob(os.path.join(root_path, "output_op", clip_name, "*.*")))
  

def download_youtube():
  #requires that YOUTUBE_ID and connected variables are set. 
  #path_o = os.path.join(root_path, "input_original")
  print(generic_file_name, specific_file_name)

  # skip downloading a youtube video we have already 
  print(os.getcwd())
  if os.path.isfile(specific_file_name):
    print("The video was already on GDrive")
    print("Renaming it from ", specific_file_name, " to: ", generic_file_name)
    shutil.copyfile(specific_file_name, generic_file_name)
  else:
    # download from Youtube
    !youtube-dl -f 'bestvideo[ext=mp4]' --output "$root_path/input_original/youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

  print("The video is downloaded ")
  print("============================== ")
  print(" ")
  return None



def check_video_speed():
  
  # there are now two different ways of dealing with filenames. one for ffprobe
  # that use os.chdir(root_path), and then all the other parts, 
  # where root_path is not used as working directory.
  # It is bad practice, but the easy way to make it reliable. $root_path/ did fail.

  ## Check for the speed of the video -------------------
  os.chdir(root_path)
  frames_per_s = default_fps            # if everything fails there is a default value
  if os.path.isfile("input_original/youtube.mp4"):
    print("Found the video for speed check.")
    text = subprocess.getoutput('ffprobe -v 0 -of csv=p=0 -select_streams v:0 -show_entries stream=r_frame_rate "input_original/youtube.mp4" ')
    print(text)
    frames, seconds = text.split("/")
    frames_per_s = int(frames) / int(seconds)
  else:
    print("Can't find the video for speed check.")

  print("Number of frames per second: ", frames_per_s)
  print("\n")
  if frames_per_s < 20:
    print("Using default values for fps.")
    multiplikator_for_images_per_second = frames_per_s / save_frames_per_second
  else:
    print("Using fps found in the video.")
    multiplikator_for_images_per_second = default_fps / save_frames_per_second
  # change the folder back.
  os.chdir(original_working_dir)
  print(os.getcwd())
  return multiplikator_for_images_per_second
  
def preprocess_video():
  print("---------------- \n Preprocessing the Video.")
  os.chdir(original_working_dir)
  print(os.getcwd())
  # cut the part of video that we want to use ---------------
  !ffmpeg -y -loglevel info -i "$root_path/input_original/youtube.mp4" -ss $start_mmss -t $duration "$root_path/input_original/video.mp4"

  # makes a copy of the original video from youtube. -------------
  if os.path.isfile(generic_file_name):
    print("creating a video file named after youtube_id", YOUTUBE_ID)
    import shutil
    #os.rename(generic_file_name, specific_file_name)
    shutil.copyfile(generic_file_name, specific_file_name)

def run_openpose():
  print("============================== \n")
  print("OpenPose")
  print("Creating the stickfigure video and json")
  print(" ")
  os.chdir(original_working_dir)
  print(os.getcwd())

  # detect poses 
  !cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/input_original/video.mp4" --write_json "$root_path/output_op/$clip_name/" --display 0  --write_video "$root_path/output_op/$clip_name/openpose.avi"

  # convert the result into MP4
  print(" ")
  print("============================== ")
  print("Converting the stickfigure video to mp4 ")
  print(" ")
  !ffmpeg -y -loglevel info -i "$root_path/output_op/$clip_name/openpose.avi" "$root_path/output_op/$clip_name/openpose.mp4" 
  print(" ")
  print("Creating single frame images ")
  print("============================== ")
  print(" ")

  ## Get only a selection of frames for labeling
  ## TODO: the numbering should match the numbering of the json files
  # save_frames_per_second = save_frames_per_second 
  !ffmpeg -y -loglevel info -i "$root_path/output_op/$clip_name/openpose.mp4" -s $size_of_single_frame -r $save_frames_per_second -f image2 "$root_path/output_op/$clip_name/frame-%06d.jpeg" 
  print(" ")
  print("============================== ")
  print("Go to folder ", root_path, "/output_op/", clip_name)


def rename_jpgs():
  os.chdir(original_working_dir)
  print(os.getcwd())
  #find all single frame jpg and change their number to match the json files
  paths_jpg = glob.glob(os.path.join(root_path, "output_op", clip_name, "f*.jpeg"))
  paths_jpg = sorted(paths_jpg, reverse=True)
  print("there are ", len(paths_jpg), " jpg files to rename")
   
  for frame in paths_jpg:
    _ , frame_nr, extention = split(r'[-\.]' , frame)    # the only thing we really need is the frame_nr
    # print(frame_nr, extention)                         # check
    # output should be 6 digits string padded with zeros if needed
    frame_number_str = str(round(int(frame_nr) * multiplikator_for_images_per_second, ndigits = None)).zfill(6)   
    new_name_frame = os.path.join(root_path, "output_op", clip_name, "frame-"+ frame_number_str +"." + extention )
    # print(new_name_frame)                              # check
    os.rename(frame, new_name_frame)
    
  print(" ")
  print("jpgs are renamed ")
  print("============================== ")
  print(" ")


def rename_json():
  ## Better names for JSON  
  #find all json files and change the name
  os.chdir(original_working_dir)
  print(os.getcwd())
  paths_json = glob.glob(os.path.join(root_path, "output_op", clip_name, "*.json"))
  paths_json = sorted(paths_json, reverse=True)
  print("there are ", len(paths_json), " json files to rename")

  for old_file in paths_json:
    frame_number = old_file[-21:-15]    # counting backwards is safest, because folder names change
    new_file_name = os.path.join(root_path, "output_op", clip_name) + "/frame-" + frame_number + ".json"
    # print(old_file, " ", new_file_name) # just a check
    os.rename(old_file, new_file_name)
  
  print(" ")
  print("json are renamed")
  print("============================== ")
  print(" ")



In [55]:
# download_youtube()

# Run the whole thing

In [56]:

### This runs all the functions
for i in range(len(youtube_list)):
  print(" ")
  print("================================================================================")
  print(" ")
  YOUTUBE_ID, clip_name, start_mmss, duration = youtube_list[i]
  print("Processing: ", YOUTUBE_ID, " ", clip_name)
  print(" ")
  print("================================================================================")
  YouTubeVideo(YOUTUBE_ID)
  prepare_folders_and_delete_outputs()
  download_youtube()
  multiplikator_for_images_per_second = check_video_speed()
  preprocess_video()
  run_openpose()
  rename_jpgs()
  rename_json()

print(" ")
print("*****************************************************************************************")
print("       DONE        ")
print("*****************************************************************************************")
print(" \n \n \n \n")


 
 
Processing:  Xm8w6L1bFoQ   Suzie_Q_ToeTap_HookStep_GhettoFab
 
Created a folder for the Openpose outputs :  /gdrive/My Drive/DSR/DanceApp /output_op/ Suzie_Q_ToeTap_HookStep_GhettoFab
Deleting old files from the  Suzie_Q_ToeTap_HookStep_GhettoFab  folder

 Now the folder contains only these files:
[]
/gdrive/My Drive/DSR/DanceApp/input_original/youtube.mp4 /gdrive/My Drive/DSR/DanceApp/input_original/9jxMHhuvEyw.mp4
/content
The video was already on GDrive
Renaming it from  /gdrive/My Drive/DSR/DanceApp/input_original/9jxMHhuvEyw.mp4  to:  /gdrive/My Drive/DSR/DanceApp/input_original/youtube.mp4
The video is downloaded 
 
Found the video for speed check.
25/1
Number of frames per second:  25.0


Using fps found in the video.
/content
---------------- 
 Preprocessing the Video.
/content
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=ha

In [57]:
from IPython.display import Audio, display
display(Audio('/gdrive/My Drive/Colab Notebooks/A-Tone-His_Self-1266414414.mp3', autoplay=True))
#http://soundbible.com/grab.php?id=1815&type=mp3